In [1]:
# NS
#optiver-ver (for ensemble)

#optiver-ver-nn-modeling/inference-v1
#optiver-ver-rnn-modeling/inference-v1
#optiver-ver-lgb-modeling/inference-v1


import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
# import lightgbm as lgb
# import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy.stats import hmean

from sklearn.ensemble import HistGradientBoostingRegressor
import itertools
import pickle
import joblib
from itertools import combinations
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import concatenate,Dropout
import pickle
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import Callback
import random
from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector

from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D, Activation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import MaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import Add

#---------------------------------------------------------------------- setup dashboard ------------------------------------------------------------

kaggle              = True
is_inference        = True
load_models         = True
run_pipeline        = False
train_models        = False
is_rnn              = True   #3
online_learning     = True




#public-validation
# dates_train = [0,390]
# dates_test = [391,480]

#full-inference
dates_train = [0,480]
dates_test = [-1,-1]


num_models ={'rnn':2}


if kaggle:
    train_path = "/kaggle/input/optiver-trading-at-the-close/train.csv"
    models_path = "/kaggle/input/optiver-rnn-just-imb-models/"
else:
    models_path = "optiver-inference-models/"
    train_path = "train.csv"

if dates_train[1]!=480:
    models_path = "vals-258/"
#---------------------------------------------------------------------- setup dashboard ------------------------------------------------------------

simulation_path = "vals-258/"

In [2]:

rnn_ep         = 1
rnn_lr         = 0.001
rnn_bs         = 2**12
window_size    = 3

In [3]:
#@title read train

colab = False
if colab:
  from google.colab import drive
  drive.mount('/content/drive')
  train_path = '/content/drive/My Drive/optiver/train.csv'
  models_path = "/content/drive/My Drive/optiver/inference/optiver-rnn-ji/"
else:
    if not kaggle:
        from public_timeseries_testing_util import MockApi

pd.set_option('mode.chained_assignment', None)


def convert_price_cols_float32(df):

    # Columns containing 'price'
    price_columns = [col for col in df.columns if 'price' in col]
    df[price_columns] = df[price_columns].astype('float32')

    # Columns containing 'wap'
    wap_columns = [col for col in df.columns if 'wap' in col]
    df[wap_columns] = df[wap_columns].astype('float32')

    return df

train = pd.read_csv(train_path).drop(['row_id', 'time_id'], axis = 1)
nan_count = train['target'].isna().sum()
print(f"The 'target' column has {nan_count} NaN values.")

target_median = train['target'].median()
train['target'].fillna(target_median, inplace=True)

print(f"converting prices columns to float32 values.")
train = convert_price_cols_float32(train)
# ----------------------------- Reading train data -------------------------

The 'target' column has 88 NaN values.
converting prices columns to float32 values.


In [4]:
#@title functions

def split_by_date(df, dates):

    df_start, df_end = dates
    df = df[(df['date_id'] >= df_start) & (df['date_id'] <=df_end)].reset_index(drop=True)

    return df



def lag_function(df, columns_to_lag, numbers_of_days_to_lag):

    df_indexed = df.set_index(['stock_id', 'seconds_in_bucket', 'date_id'])

    for column_to_lag in columns_to_lag:
        for number_days_to_lag in numbers_of_days_to_lag:
            df_indexed[f'lag{number_days_to_lag}_{column_to_lag}'] = df_indexed.groupby(level=['stock_id', 'seconds_in_bucket'])[column_to_lag].shift(number_days_to_lag)

    df_indexed.reset_index(inplace=True)

    return df_indexed




def compute_imbalances(df_, columns, prefix = ''):
    """Computes the differences and imbalances for pairs of columns and stores them in the DataFrame."""
    df = df_.copy()
    for col1, col2 in combinations(columns, 2):

        # Sort the columns lexicographically to ensure consistent ordering
        col1, col2 = sorted([col1, col2])

        # Compute imbalance directly without creating a temporary difference column
        total = df[col1] + df[col2]
        imbalance_column_name = f'{col1}_{col2}_imb{prefix}'

        # Ensure we don't divide by zero
        df[imbalance_column_name] = (df[col1] - df[col2]).divide(total, fill_value=np.nan)

    return df



def save_pickle(data, file_path):

    # Create the directory if it doesn't exist
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save the pickle file
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

    print(f"Data saved to {file_path}")
    #example: save_pickle(all_data, 'k8/all_data.pkl')

def load_pickle(file_path):

    # Load and return the data from the pickle file
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        return data
    else:
        raise FileNotFoundError(f"No such file: {file_path}")

    #example: loaded_data = load_pickle('k8/all_data.pkl')



In [5]:
#@title pipeline

raw_cols          = ['imbalance_size','matched_size','bid_size','ask_size','reference_price','far_price','near_price','bid_price','ask_price','wap','imbalance_buy_sell_flag']

columns_prices    = ['reference_price','far_price','near_price','bid_price','ask_price','wap']
columns_4prices   = ['reference_price','bid_price','ask_price','wap']

columns_sizes     = ['imbalance_size','matched_size','bid_size','ask_size']
columns_flag      = ['imbalance_buy_sell_flag']


num_of_target_lags  = 3
target_lags         = list(range(1,num_of_target_lags+1))





def feature_pipeline(df):

    if df.empty:
        return pd.DataFrame()


    df = compute_imbalances(df, columns_sizes,prefix='_sz_')
    df = compute_imbalances(df, columns_prices,prefix = '_pr_')


    print(f"lagging target column for {len(target_lags)} lags.")
    df = lag_function(df, ['target'], target_lags)



    print("Done...")

    return df

In [6]:

def make_predictions(models, X_test,model = 'nn'):
    if model == 'nn':
        all_predictions = [model.predict(X_test, batch_size=16384) for model in models]
    if model == 'lgb' or model == 'xgb' or model == 'cat':
        all_predictions = [model.predict(X_test) for model in models]
    prediction = np.mean(all_predictions, axis=0)
    return prediction

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

class BestScoresCallback(Callback):
    def __init__(self):
        super().__init__()
        self.best_train_loss = float('inf')
        self.best_val_loss = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        train_loss = logs.get('loss', float('inf'))
        val_loss = logs.get('val_loss', float('inf'))

        if train_loss < self.best_train_loss:
            self.best_train_loss = train_loss
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss

    def on_train_end(self, logs=None):
        print(f"Best training loss: {self.best_train_loss}, Best validation loss: {self.best_val_loss}")


In [7]:
#@title RNN second pass

def precompute_sequences(stock_data, window_size, rnn_numerical_features, rnn_categorical_features):
    # Convert DataFrame columns to NumPy arrays
    stock_data_num = stock_data[rnn_numerical_features].values
    stock_data_cat = stock_data[rnn_categorical_features].values

    # Pre-compute all sequences
    all_sequences_num = [stock_data_num[max(0, i - window_size + 1):i + 1] for i in range(len(stock_data))]
    all_sequences_cat = [stock_data_cat[max(0, i - window_size + 1):i + 1] for i in range(len(stock_data))]

    # Add padding if necessary
    padded_sequences_num = [np.pad(seq, ((window_size - len(seq), 0), (0, 0)), 'constant') for seq in all_sequences_num]
    padded_sequences_cat = [np.pad(seq, ((window_size - len(seq), 0), (0, 0)), 'constant') for seq in all_sequences_cat]

    # Combine numerical and categorical features
    combined_sequences = np.array([np.concatenate([num, cat], axis=-1)
                                   for num, cat in zip(padded_sequences_num, padded_sequences_cat)])

    # Extract targets
    targets = stock_data['target'].values

    return combined_sequences, targets

def get_sequence(precomputed_data, time_step):
    combined_sequences, targets = precomputed_data
    return combined_sequences[time_step], targets[time_step]



def create_batches(data, window_size, rnn_numerical_features, rnn_categorical_features, max_time_steps=55):

    grouped = data.groupby(['stock_id', 'date_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(max_time_steps):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0:
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    return all_batches, all_targets




def compute_last_sequence(group, window_size, rnn_numerical_features, rnn_categorical_features):
    # Convert DataFrame columns to NumPy arrays
    stock_data_num      = group[rnn_numerical_features].values
    stock_data_cat      = group[rnn_categorical_features].values
    stock_data_target   = group['target'].values

    # Find the index of the target second
    target_index = len(group) - 1

    # Extract the sequence for the target index
    sequence_num = stock_data_num[max(0, target_index - window_size + 1):target_index + 1]
    sequence_cat = stock_data_cat[max(0, target_index - window_size + 1):target_index + 1]

    # Add padding if necessary
    padded_sequence_num = np.pad(sequence_num, ((window_size - len(sequence_num), 0), (0, 0)), 'constant')
    padded_sequence_cat = np.pad(sequence_cat, ((window_size - len(sequence_cat), 0), (0, 0)), 'constant')

    # Combine numerical and categorical features
    combined_sequence = np.concatenate([padded_sequence_num, padded_sequence_cat], axis=-1)

    # Extract target
    target = stock_data_target[-1]

    return combined_sequence, target


def create_last_batches(data, window_size, rnn_numerical_features, rnn_categorical_features):

    grouped = data.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in grouped:
        # Compute the sequence for the last data point in the current group
        last_sequence, last_target = compute_last_sequence(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Check if the sequence is valid (i.e., not empty)
        if last_sequence.size > 0:
            all_batches.append(last_sequence)
            all_targets.append(last_target)

    return all_batches, all_targets





def second_pass_for_rnn(df, rnn_numerical_features, rnn_categorical_features, window_size, is_inference=False):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians

    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    if is_inference:
        df_copy.fillna(rnn_medians, inplace=True)
        df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])

    # Preprocess Data
    df_copy['seconds_in_bucket'] = df_copy['seconds_in_bucket'] / 10
    df_copy['imbalance_buy_sell_flag'] += 1

    if is_inference:
        df_copy_batches, df_copy_targets = create_last_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)
    else:
        df_copy_batches, df_copy_targets = create_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)

    df_copy_batches = np.array(df_copy_batches)
    df_copy_targets = np.array(df_copy_targets)


    return df_copy_batches, df_copy_targets


def online_pass_for_rnn(df, rnn_numerical_features, rnn_categorical_features, window_size):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians

    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    df_copy.fillna(rnn_medians, inplace=True)
    df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])

    # Preprocess Data
    df_copy['seconds_in_bucket'] = df_copy['seconds_in_bucket'] / 10
    df_copy['imbalance_buy_sell_flag'] += 1


    grouped = df_copy.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(55):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0:
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    df_batches = np.array(all_batches)
    df_targets = np.array(all_targets)


    return df_batches, df_targets



In [8]:
#@title RNN model

def apply_conv_layers(input_layer, kernel_sizes, filters=16, do_ratio=0.5):
    conv_outputs = []

    for kernel_size in kernel_sizes:
        conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(input_layer)
        conv_layer = BatchNormalization()(conv_layer)
        conv_layer = Dropout(do_ratio)(conv_layer)

        shortcut = conv_layer

        conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(conv_layer)
        conv_layer = BatchNormalization()(conv_layer)
        conv_layer = Activation('relu')(conv_layer)

        # Add the output of the first Conv1D layer
        conv_layer = Add()([conv_layer, shortcut])
        conv_outputs.append(conv_layer)


    concatenated_conv = Concatenate(axis=-1)(conv_outputs)
    flattened_conv_output = Flatten()(concatenated_conv)

    return flattened_conv_output

def create_rnn_model_with_residual(window_size, numerical_features, initial_learning_rate=0.001):

    categorical_features = 'seconds_in_bucket'
    categorical_uniques  = { 'seconds_in_bucket' : 55}
    embedding_dim        = {'seconds_in_bucket' : 10}

    input_layer = Input(shape=(window_size, len(numerical_features) + 1), name="combined_input")

    # Split the input into numerical and categorical parts
    numerical_input = Lambda(lambda x: x[:, :, :-1], name="numerical_part")(input_layer)
    categorical_input = Lambda(lambda x: x[:, :, -1:], name="categorical_part")(input_layer)

    first_numerical = Lambda(lambda x: x[:, 0])(numerical_input)


    # diffrentiate layers
    def create_difference_layer(lag):
        return Lambda(lambda x: x[:, lag:, :] - x[:, :-lag, :], name=f"difference_layer_lag{lag}")

    difference_layers = []
    for lag in range(1, window_size):
        diff_layer = create_difference_layer(lag)(numerical_input)
        padding = ZeroPadding1D(padding=(lag, 0))(diff_layer)  # Add padding to the beginning of the sequence
        difference_layers.append(padding)
    combined_diff_layer = Concatenate(name="combined_difference_layer")(difference_layers)


    # Embedding for categorical part
    vocab_size, embedding_dim = categorical_uniques[categorical_features], embedding_dim[categorical_features]
    embedding = Embedding(vocab_size, embedding_dim, input_length=window_size)(categorical_input)
    embedding = Reshape((window_size, -1))(embedding)

    first_embedding = Lambda(lambda x: x[:, 0])(embedding)

    # Concatenate numerical input and embedding
    # conv_input = concatenate([enhanced_numerical_input, embedding], axis=-1)

    kernel_sizes = [2,3]
    do_ratio = 0.4

    flattened_conv_output = apply_conv_layers(numerical_input, kernel_sizes, do_ratio=do_ratio)
    flattened_conv_output_cat = apply_conv_layers(embedding, kernel_sizes, do_ratio=do_ratio)
    flattened_conv_output_diff = apply_conv_layers(combined_diff_layer, kernel_sizes, do_ratio=do_ratio)


    dense_output = Concatenate(axis=-1)([flattened_conv_output,flattened_conv_output_cat,flattened_conv_output_diff, Reshape((-1,))(combined_diff_layer),first_numerical,first_embedding])

    dense_sizes = [512, 256, 128, 64, 32]
    do_ratio = 0.5
    for size in dense_sizes:
        dense_output = Dense(size, activation='swish')(dense_output)
        dense_output = BatchNormalization()(dense_output)
        dense_output = Dropout(do_ratio)(dense_output)

    # Output layer
    output = Dense(1, name='output_layer')(dense_output)

    # Learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=10000,
        decay_rate=0.7,
        staircase=True)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss="mean_absolute_error")

    return model

In [9]:
#@title runnig pipeline

excluded_columns = ['row_id', 'date_id', 'time_id', 'target','stock_return']

if run_pipeline:

    train_eng = feature_pipeline(train)

    if is_rnn:
        excluded_columns = excluded_columns + ['stock_id']

        features = [col for col in train_eng.columns if col not in excluded_columns]
        rnn_categorical_features =  ['seconds_in_bucket']
        rnn_numerical_features = [feat for feat in features if feat not in rnn_categorical_features]
        print("we have {} numerical and {} categorical".format(len(rnn_numerical_features),len(rnn_categorical_features)))


        rnn_scaler = StandardScaler()
        rnn_medians = train_eng.median()

        train_eng.fillna(rnn_medians, inplace=True)
        train_eng[rnn_numerical_features] = rnn_scaler.fit_transform(train_eng[rnn_numerical_features])


        rnn_all_data = {
            "rnn_scaler": rnn_scaler,
            "rnn_medians": rnn_medians,
            "rnn_categorical_features": rnn_categorical_features,
            "rnn_numerical_features": rnn_numerical_features
        }

        save_pickle(rnn_all_data, f'{models_path}rnn_all_data.pkl')
        print("Pipline Done!")


    train_data       = split_by_date(train_eng, dates_train)
    test_data        = split_by_date(train_eng, dates_test)
    print("number of dates in train = {} , number of dates in test {}".format (train_data['date_id'].nunique(),test_data['date_id'].nunique()))

    cleaning = False
    if cleaning:
        import gc
        #del train
        del train_eng
        gc.collect()




lagging target column for 3 lags.
Done...
we have 35 numerical and 1 categorical
Data saved to vals-258/rnn_all_data.pkl
Pipline Done!
number of dates in train = 6 , number of dates in test 6


In [10]:
#@title TPU
try:
    # Create a TPUClusterResolver
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    # Connect to the TPU cluster
    tf.config.experimental_connect_to_cluster(tpu)
    # Initialize the TPU system
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # Create a TPUStrategy for distributed training
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    tpu_strategy = None  # No TPU found


In [11]:
#@title prepare train models
if train_models:

    if test_data.empty:
        print("--------------test data is empty so adjusting the last date for test-----------------")
        test_data = train_data.query("date_id == 480").copy()

    if is_rnn:

        train_batches, train_targets = second_pass_for_rnn(train_data, rnn_numerical_features, rnn_categorical_features, window_size)
        test_batches, test_targets  = second_pass_for_rnn(test_data, rnn_numerical_features, rnn_categorical_features, window_size)
        print(f"train batches shape:{train_batches.shape}")


Processing groups: 100%|██████████| 1200/1200 [00:09<00:00, 127.26it/s]


train batches shape:(66000, 3, 36)


In [12]:
#@title training models
# train_data = flatten_outliers(train_data, 'target', lower_quantile=0.01, upper_quantile=0.99)
if train_models:

        directory = os.path.dirname(models_path)
        if not os.path.exists(directory):
            os.makedirs(directory)


        if is_rnn:

            callbacks = [BestScoresCallback()]  # Always include BestScoresCallback
            if dates_train[1] != 480:
                early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
                callbacks.append(early_stopping)

            if True or tpu_strategy:
                # with tpu_strategy.scope():

                    rnn_models = []
                    for i in range(num_models['rnn']):

                        print(f"Training rnn model {i+1} out of {num_models['rnn']} with seed {42+i}")
                        print("---------------------------------------")
                        set_all_seeds(42+i)

                        rnn_model = create_rnn_model_with_residual(window_size, rnn_numerical_features, initial_learning_rate=rnn_lr)
                        history = rnn_model.fit(train_batches, train_targets, validation_data=(test_batches, test_targets), epochs=rnn_ep, batch_size=rnn_bs, callbacks=callbacks)
                        print("---------------------------------------")
                        rnn_model.save(f'{models_path}rnn_model_seed_{i}.h5')
                        rnn_models.append(rnn_model)

                    predictions =  make_predictions(rnn_models, test_batches,model = 'nn')
                    print(f"Ensemble Mean Absolute Error: {mean_absolute_error(test_targets, predictions):.4f}")


Training rnn model 1 out of 2 with seed 42
---------------------------------------
17/17 [==============================] - 17s 332ms/step - loss: 6.1802 - val_loss: 6.0470
Best training loss: 6.180210113525391, Best validation loss: 6.046993732452393
---------------------------------------


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training rnn model 2 out of 2 with seed 43
---------------------------------------
17/17 [==============================] - 16s 353ms/step - loss: 6.1880 - val_loss: 6.0473
Best training loss: 6.180210113525391, Best validation loss: 6.046993732452393
---------------------------------------
5/5 [==============================] - 2s 159ms/step
Ensemble Mean Absolute Error: 6.0466


In [13]:
#@title timeseries functions

def clean_format(df):
    df['target'] = df['target'].astype('float64')
    for feat in ['stock_id','date_id','seconds_in_bucket']:
        df[feat] = df[feat].astype('int64')
    return df

def clean_up(df):
    df = df[['stock_id','revealed_date_id','seconds_in_bucket','revealed_target']].rename(columns={'revealed_date_id': 'date_id', 'revealed_target': 'target'})
    df['target'].fillna(-0.06020069, inplace=True)
    df = clean_format(df)
    return df

def manage_buffer(buffer, df, max_lag):

    # df['iteration'] = df['iteration'].max()  # Ensure the iteration number is consistent in the new df chunk
    if buffer.empty:
        return df.copy()
    elif buffer['iteration'].nunique() < max_lag:
        return pd.concat([buffer, df], ignore_index=True)
    else:
        oldest_iteration = buffer['iteration'].min()
        buffer = buffer[buffer['iteration'] > oldest_iteration]
        return pd.concat([buffer, df], ignore_index=True)


def append_target_lags(df, target_buffer, lags):

    column_to_lag = 'target'
    df_lagged = df.copy()

    for lag in lags:

        temp_df = target_buffer.copy()
        temp_df['date_id'] = temp_df['date_id'] + lag

        temp_df.rename(columns={column_to_lag: f'lag{lag}_{column_to_lag}'}, inplace=True)

        df_lagged = pd.merge(df_lagged, temp_df[['stock_id', 'date_id', 'seconds_in_bucket', f'lag{lag}_{column_to_lag}']],
                             on=['stock_id', 'date_id', 'seconds_in_bucket'],
                             how='left')

    return df_lagged




In [14]:
if kaggle:
    import gc
    # remove train_small and train and clear the memory
    del train
    gc.collect()

In [15]:
#@title is inference
if is_inference:

    if online_learning:


        #rnn
        online_rnn_lr_rate    = 2e-4 #2e-4
        online_rnn_batch_size = 2**13 #2**12 reduced for memory
        online_rnn_epochs     = 4      #2



    if kaggle:
        import optiver2023
        env = optiver2023.make_env()
        iter_test = env.iter_test()
    else:
        env = MockApi()
        iter_test = env.iter_test()

    if load_models:



        if is_rnn:

            print("loading rnn models...")
            loaded_data          = load_pickle(f'{models_path}rnn_all_data.pkl')
            rnn_scaler               = loaded_data["rnn_scaler"]
            rnn_medians              = loaded_data["rnn_medians"]
            rnn_categorical_features = loaded_data["rnn_categorical_features"]
            rnn_numerical_features   = loaded_data["rnn_numerical_features"]

            rnn_models = []
            for i in range(num_models['rnn']):
                loaded_model = load_model(f"{models_path}rnn_model_seed_{i}.h5")
                rnn_models.append(loaded_model)
            print("Done!")



    #---------------------------buffer management ---------------------------
    max_target_lag  = num_of_target_lags
    i               = 0
    target_counter  = 0
    target_buffer   = pd.DataFrame()
    total_test      = pd.DataFrame()
    daily_online    = pd.DataFrame()
    #---------------------------buffer management ---------------------------


    for (test, revealed_targets, sample_prediction) in iter_test:


        test = convert_price_cols_float32(test)


        if test.seconds_in_bucket.iloc[0] == 0:

            #----------------------------- storing previous targets -------------------------
            revealed_targets = clean_up(revealed_targets)
            revealed_targets['iteration'] = target_counter
            target_buffer = manage_buffer(target_buffer, revealed_targets, max_target_lag)
            target_counter += 1



            if not daily_online.empty and online_learning:

                daily_online.drop(columns='target', inplace=True)
                daily_online = pd.merge(daily_online, revealed_targets[['stock_id', 'date_id', 'seconds_in_bucket', 'target']],
                                            on=['stock_id', 'date_id', 'seconds_in_bucket'],
                                            how='inner')


                if is_rnn:

                    online_batches, online_targets = online_pass_for_rnn(daily_online, rnn_numerical_features, rnn_categorical_features, window_size)

                    for rnn_model in rnn_models:
                        lr_schedule = ExponentialDecay( initial_learning_rate=online_rnn_lr_rate, decay_steps=10000, decay_rate=0.8,staircase=True)
                        optimizer = Adam(learning_rate=lr_schedule)
                        rnn_model.compile(optimizer=optimizer, loss = "mean_absolute_error")
                        rnn_model.fit(online_batches, online_targets, epochs=online_rnn_epochs, batch_size=online_rnn_batch_size)
                        K.clear_session()
                        gc.collect()

            daily_online = pd.DataFrame()
            #----------------------------- storing previous targets -------------------------


        test['target'] = 0


 
        test = compute_imbalances(test, columns_sizes,prefix='_sz_')
        test = compute_imbalances(test, columns_prices,prefix = '_pr_')

        test = append_target_lags(test, target_buffer, target_lags)

        if not daily_online.empty:
            full_data = pd.concat([daily_online[test.columns], test], ignore_index=True)
        else:
            full_data = test

        # getting back to test size
        test = full_data.iloc[-test.shape[0]:].copy()


        if test.currently_scored.iloc[0]:

            if is_rnn:
                X_test, _           = second_pass_for_rnn(full_data, rnn_numerical_features, rnn_categorical_features, window_size, is_inference=True)
                test['target'] += make_predictions(rnn_models, X_test,model='nn').flatten()



        if not kaggle:
            total_test = pd.concat([total_test, test], ignore_index=True)


        daily_online = pd.concat([daily_online, test], ignore_index=True)


        sample_prediction = pd.merge(sample_prediction.drop(columns='target'), test[['row_id', 'target']], on=['row_id'], how='left')
        sample_prediction['target'].fillna(0, inplace=True)

        env.predict(sample_prediction)
        i += 1

loading rnn models...
Done!
This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


Processing groups: 100%|██████████| 200/200 [00:01<00:00, 111.26it/s]


Epoch 1/4
2/2 [==============================] - 11s 168ms/step - loss: 5.3183
Epoch 2/4
2/2 [==============================] - 1s 154ms/step - loss: 5.3245
Epoch 3/4
2/2 [==============================] - 1s 163ms/step - loss: 5.3382
Epoch 4/4
2/2 [==============================] - 1s 165ms/step - loss: 5.3144
Epoch 1/4
2/2 [==============================] - 11s 172ms/step - loss: 5.3426
Epoch 2/4
2/2 [==============================] - 1s 154ms/step - loss: 5.2941
Epoch 3/4
2/2 [==============================] - 1s 159ms/step - loss: 5.3137
Epoch 4/4
2/2 [==============================] - 1s 153ms/step - loss: 5.3159


Processing groups: 100%|██████████| 200/200 [00:01<00:00, 112.68it/s]


Epoch 1/4
2/2 [==============================] - 11s 169ms/step - loss: 5.8347
Epoch 2/4
2/2 [==============================] - 1s 161ms/step - loss: 5.8590
Epoch 3/4
2/2 [==============================] - 1s 167ms/step - loss: 5.8248
Epoch 4/4
2/2 [==============================] - 1s 178ms/step - loss: 5.8296
Epoch 1/4
2/2 [==============================] - 11s 170ms/step - loss: 5.8536
Epoch 2/4
2/2 [==============================] - 1s 162ms/step - loss: 5.8410
Epoch 3/4
2/2 [==============================] - 1s 171ms/step - loss: 5.8234
Epoch 4/4
2/2 [==============================] - 1s 162ms/step - loss: 5.8415
